# FORMATION DU DATASET

Ici nous nous allons travailler a construire un dataset grace à une suite de fonctions pour pretraiter des tweets. Le cadre d'utilisation de ces donnée sera l'analyse de sentiment mais on peut tout aussi bien adapter ce qui est developé ici à d'autres contextes.

## Les Imports

In [ ]:
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

import re, string, random
import numpy as np

nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

stop_words = stopwords.words('english')
# les stop word sont des mots telque ‘ourselves’, ‘hers’, ‘between’, ‘yourself’, ‘but’, ... 
# des mots assez courant pour apparaitre dans n'import quel texte

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tout d'abord importons le dataset, nltk met a disposition plusieurs sets de tweets en langue anglaise avec des labels de tweet neutres, positifs et negatifs, nous utliserons ce qui est developpé dans ce notebook pour l'apprentisage de nos classifieurs.

In [ ]:
#import du dataset et tokenization

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
neutral_tweet_tokens = np.random.choice(twitter_samples.tokenized('tweets.20150430-223406.json'),size=len(negative_tweet_tokens),replace = False)

#pour l'utilisation d'un dataset non equilibré:
#neutral_tweet_tokens = twitter_samples.tokenized('tweets.20150430-223406.json')


## Le Pré-traitement

Ci-dessous quelques fonctions qui permetrons de pré-traiter les tweets.

* Nous utilison des tweet tokenisé, c'est dire que l'on passe d'une string a une liste de token, par exemple :
  * "les vache broute dans le champ" devient ["les", "vache", "broutent", "dans",  "le", "champ"]

* les tweets sont en premier lieu nettoyé des partie que nous jugeons dispensable les adresses de sites, les noms, les chiffres, les nombres, les ponctuations,... 

* Certain mots appelés stop words, sont aussi suprimés.ce sont des mot qui apparaise dans n'importe qule contexte, en francais ce serait les mot le, la, les, à, ... en englais ce sont leurs traductions ainsi que les auxiliaires utilisé pour les temps et d'autres plus specifiques à l'anglais.
(pour le detail de ces mots, regarder le contenu de la liste stop_words)

* et enfin les token restant sont lemmetizés c'est a dire dans le cas du francais la liste ["cours", "courrant", "courrir"] devient ["cour", "cour","cour"] ou bien en anglais ["run", "ran", "running"] devient ["run", "run", "run"]

Ainsi nous reduisons le dictionaire des mots utilisés sans pour autant perdre beaucoup de sens.

In [ ]:
def remove_noise(tweet_tokens, stop_words = ()): #fait le nettoyage comme expliqué plus haut

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens


def get_tweets_for_model(cleaned_tokens_list):# Nécessaire pour avoir le format requis pour les classifieurs de NLTK
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

def make_set( tweet_tokens, label="", stop_words = () ) :# nettoie les tweets tokenisés et les associe à un label
    cleaned_tokens_list = []
    for tokens in tweet_tokens:
        cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    tokens_for_model = get_tweets_for_model(cleaned_tokens_list)
    dataset = [(tweet_dict, label)for tweet_dict in tokens_for_model]
    return dataset

Les defaults de cette manière de faire, sont :
* ne prend pas en compte la position des mots dans la phrase.
* les temps des phrases et les sens associés au terminaisons des mots sont effacé.
* les sens d'expression telque "Tomber dans les pommes" peut disparaitre.
* ...


Cette manière de binariser un texte est une parmis d'autre , on pourrait aussi travailler avec des n-uple de mots, par exemple avec des couple de mots:
* "les vache broutent l'herbe " devient ["les vache", "les broutent", "les l'herbe" ,"vache broute", "vache l'herbe", "broute l'herbe"]. 

Ces manières là peuvent reduire quelques defaults plus haut mais en apporte d'autre avec eux, une dimension plus grande des données, des combinaison de mot qui n'apparaissent pratiquement jamais,...


In [ ]:
#  nettoyage et formation du dataset

positive_dataset = make_set( positive_tweet_tokens, label="Positive", stop_words = stop_words )
negative_dataset = make_set( negative_tweet_tokens, label="Negative", stop_words = stop_words )
neutral_dataset = make_set( neutral_tweet_tokens, label="Neutral", stop_words = stop_words )

dataset = positive_dataset + negative_dataset + neutral_dataset
random.shuffle(dataset)# mélange aleatoire du data set

Pour conclure nous obtenons un dataset labelisé et prêt etre utilisé avec les classifieurs nltk (Pour l'utilisation avec des classifieurs de bibliotèque plus classiques (sklearn,keras,...) quelques boucles suffisent). Bien qu'imparfait, ce format de dataset est utlisé très courramment et permet d'obtenir des resultats convenables que nous verrons dans le notebook Analyse_de_sentiment.





##Documentation Utilisée

* documentation de nltk : https://www.nltk.org/index.html
* https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk
